# Ejercicio 13: LangChain



*   **Estudiante:** José Quiros



LangChain es un _framework_ de código abierto diseñado para facilitar el desarrollo de aplicaciones que combinan modelos de lenguaje LLMs con datos, herramientas externas y memoria. Está especialmente pensado para construir aplicaciones complejas basadas en IA, como sistemas _Retrieval-Augmented Generation_, asistentes conversacionales inteligentes, agentes autónomos y sistemas con razonamiento compuesto.

## Parte 1: Carga y preprocesamiento del corpus

In [ ]:
!pip install dotenv

In [ ]:
import os
import json
from dotenv import load_dotenv

# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Crear archivo kaggle.json desde variables de entorno
kaggle_token = {
    "username": os.getenv("KAGGLE_USERNAME"),
    "key": os.getenv("KAGGLE_KEY")
}

# Crear carpeta de configuración para kaggle en /root/.kaggle/
kaggle_path = os.path.join(os.path.expanduser("~"), ".kaggle")
os.makedirs(kaggle_path, exist_ok=True)

# Guardar kaggle.json
with open(os.path.join(kaggle_path, "kaggle.json"), "w") as f:
    json.dump(kaggle_token, f)

# Cambiar permisos (evita errores de seguridad)
os.chmod(os.path.join(kaggle_path, "kaggle.json"), 0o600)

# Importar kaggle después de haber creado kaggle.json
import kaggle


In [ ]:
dataset = "rajneesh231/lex-fridman-podcast-transcript"
path = "../data/13langchain"

kaggle.api.dataset_download_files(dataset, path=path, unzip=True)

Dataset URL: https://www.kaggle.com/datasets/rajneesh231/lex-fridman-podcast-transcript


In [ ]:
import pandas as pd
file_path = os.path.join(path, "podcastdata_dataset.csv")  # ajusta nombre si difiere
df = pd.read_csv(file_path)
df

,id,guest,title,text
0,1,Max Tegmark,Life 3.0,"As part of MIT course 6S099, Artificial Genera..."
1,2,Christof Koch,Consciousness,As part of MIT course 6S099 on artificial gene...
2,3,Steven Pinker,AI in the Age of Reason,"You've studied the human mind, cognition, lang..."
3,4,Yoshua Bengio,Deep Learning,What difference between biological neural netw...
4,5,Vladimir Vapnik,Statistical Learning,The following is a conversation with Vladimir ...
...,...,...,...,...
314,321,Ray Kurzweil,"Singularity, Superintelligence, and Immortality","By the time he gets to 2045, we'll be able to ..."
315,322,Rana el Kaliouby,"Emotion AI, Social Robots, and Self-Driving Cars","there's a broader question here, right? As we ..."
316,323,Will Sasso,"Comedy, MADtv, AI, Friendship, Madness, and Pr...",Once this whole thing falls apart and we are c...
317,324,Daniel Negreanu,Poker,you could be the seventh best player in the wh...


In [ ]:
! pip install langchain

In [ ]:
from langchain.schema import Document

# Convertir cada fila en un Document
documents = [
    Document(
        page_content=row["text"],
        metadata={"id": row["id"], "guest": row["guest"], "title": row["title"]},
    )
    for _, row in df.iterrows()
]
#documents

## Parte 2: Segmentación y embeddings

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(documents)

In [ ]:
!pip install -U langchain-community

In [ ]:
!pip install -U langchain-huggingface

In [ ]:
!pip install faiss-cpu

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import faiss

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
texts = [chunk.page_content for chunk in chunks]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Parte 3: Indexación en FAISS

In [ ]:
vectorstore = FAISS.from_texts(texts, embeddings)
vectorstore.save_local("index_13langchain")

In [ ]:
import google.generativeai as genai
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
for m in genai.list_models():
    print(m.name, m.supported_generation_methods)

models/embedding-gecko-001 ['embedText', 'countTextTokens']
models/gemini-1.0-pro-vision-latest ['generateContent', 'countTokens']
models/gemini-pro-vision ['generateContent', 'countTokens']
models/gemini-1.5-pro-latest ['generateContent', 'countTokens']
models/gemini-1.5-pro-002 ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-pro ['generateContent', 'countTokens']
models/gemini-1.5-flash-latest ['generateContent', 'countTokens']
models/gemini-1.5-flash ['generateContent', 'countTokens']
models/gemini-1.5-flash-002 ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-flash-8b ['createCachedContent', 'generateContent', 'countTokens']
models/gemini-1.5-flash-8b-001 ['createCachedContent', 'generateContent', 'countTokens']
models/gemini-1.5-flash-8b-latest ['createCachedContent', 'generateContent', 'countTokens']
models/gemini-2.5-pro-preview-03-25 ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/ge

## Parte 4: Creación de la cadena de recuperación

In [ ]:
!pip install langchain-google-genai

### 4.1. Creación de la cadena de recuperación con RetrievalQA

In [ ]:
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI

# Inicializar Gemini LLM
llm = ChatGoogleGenerativeAI(model="models/gemini-1.5-flash", temperature=0)

# Crear retriever desde tu índice vectorial (FAISS o Chroma)
retriever = vectorstore.as_retriever()

# Cadena de pregunta-respuesta
qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)

# Ejecutar una pregunta
response = qa_chain.invoke({"query":"What is AGI Artificial General Intelligence?"})
print(f">  Query: {response['query']}")
print(f"<> Respuesta: {response['result']}")

>  Query: What is AGI Artificial General Intelligence?
<> Respuesta: Based on the provided text, AGI, or Artificial General Intelligence, is a type of AI with the ability to learn and adapt to new environments.  However, it's unclear if a machine with AGI would necessarily have consciousness.  The text also notes that the term is widely used and understood, even though its precise definition is debated, with some considering it simply very smart AI surpassing human intelligence, while others equate it to human-like intelligence.


### 4.2. Creación de la cadena de recuperación con RetrievalQA y PromptTemplate
Se implementa un prompt para crear un RAG

In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
Usa solo el siguiente contexto para responder a la pregunta.
Si la respuesta no está explícita en el contexto, responde exactamente:
"No encontré información suficiente en el corpus."

Contexto:
{context}

Pregunta: {question}
Respuesta:
"""
)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

result = qa_chain.invoke({"query": "¿Qué es AGI Artificial General Intelligence?"})
print(result["result"])



No encontré información suficiente en el corpus.


### 4.3. Creación de la cadena de recuperación con RetrievalQA, para recuperación de los documentos fuentes

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    return_source_documents=True
)

result = qa_chain.invoke({"query": "¿Qué es AGI Artificial General Intelligence?"})

print(f">  Query: {result['query']}")
print(f"<> Respuesta: {result['result']}")
print(f">> Documentos Fuente: ")
for i in result["source_documents"]:
    print(f"\t ID:{i.id}")
    print(f"\t Contenido:\b\t{i.page_content}")

>  Query: ¿Qué es AGI Artificial General Intelligence?
<> Respuesta: Based on the provided text, AGI (Artificial General Intelligence) is a term used to describe artificial intelligence that reaches or surpasses human-level intelligence.  However, the text also notes that the term is not well-defined and that human intelligence itself is not truly "general" but rather highly specialized.  The text mentions AIXI as an example of a theoretical system that could be considered truly general intelligence within the constraints of computational theory.  There's no single agreed-upon test to definitively determine if AGI has been achieved.
>> Documentos Fuente: 
	 ID:3efcb5c3-8c53-4dfd-a3ec-4425d9b948e7
	 Contenido:	general intelligence, artificial intelligence, only refers to if you achieve human level or a subhuman level, but quite broad, is it also general intelligence? So we have to distinguish, or it's only super human intelligence, general artificial intelligence. Is there a test in yo

### Guardar índice de FAISS en local

In [ ]:
import shutil
from google.colab import files

# Comprime la carpeta en un .zip
shutil.make_archive("index_langchain", 'zip', "index_13langchain")

# Descarga al navegador
files.download("index_langchain.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>